In [1]:
#Se importan las librerias necesarias
import oommfc as mc
import discretisedfield as df
import micromagneticmodel as mm
import random
import numpy as np

#Funcion para minimizar la energía del sistema
md = mc.MinDriver()

#Función para llevar a cabo el proceso de histerésis
hd = mc.HysteresisDriver()

random.seed(2) #Con esto aseguramos reproducibilidad

In [2]:
#Defimos funciones para crear una subregion.
#xmin es el punto minimo en x de la subregion
#ymin es el punto minimo en y de la subregion
#zmin es el punto minimo en z de la subregion
#xmin es el punto maximo en x de la subregion
#ymin es el punto maximo en y de la subregion
#ymin es el punto maximo en y de la subregion
#subregions es una diccionario en la cual se guarda la subregion creada
#i es el número de la subregión creada
#Devuelve la subregion creada para un xmin,ymin,zmin,xmax,ymax y zmax dados.
def subreg(xmin,xmax,ymin,ymax,zmin,zmax,subregions,i):
    #Punto minimo de la subregion
    p1 = (xmin, ymin, zmin)
    #Punto maximo de la subregion
    p2 = (xmax, ymax, zmax)
    #Creacion de la region con discretisedfield
    region1 = df.Region(p1=p1, p2=p2)
    #Nombre de la región
    r='region{}'.format(i)
    #Se guarda la subregion
    subregions[r] = region1
    #Devuelve la subregion guardada
    return subregions

#Funcion para una region de 3 capas
#Recibe como parametro los valores minimos en los ejes x,y,z de la región en general
#Recibe como parametro el espesor del material no magnetico y el espesor del material magnetico
#Recibe como parametro "celda", la cual define en cuantos cuadros se puede dividir la region. Por definicion es 0.1nm
#La función devuelve la tricapa y las subregiones creadas.
def region(xminr,yminr,zminr, espesor_ferromagnetico, espesor_no_magnetico,celda=0.1e-9):  
    #Para la region 1
    xmin1=xminr
    ymin1=yminr
    zmin1=zminr
    zmax1=espesor_ferromagnetico

    #Para la region 2
    xmin2=xminr
    ymin2=yminr
    zmin2=zmax1 
    zmax2=zmin2 + espesor_no_magnetico
    
    #Para la region 3 
    xmin3=xminr
    ymin3=yminr
    zmin3=zmax2
    zmax3=zmin3 + espesor_ferromagnetico
    zmaxr=zmax3
    
    #Maximo region en general
    xmaxr=espesor_ferromagnetico
    ymaxr=espesor_ferromagnetico
    
    #Maximos en region 1
    xmax1=xmaxr
    ymax1=ymaxr
     
    #Maximos en region 2
    xmax2=xmaxr
    ymax2=ymaxr
     
    #Maximos en region 3
    xmax3=xmaxr
    ymax3=ymaxr
    
    #defino una diccionario vacio para las subregiones
    subregions={}
    
    #Creo cada una de las subregiones
    subregions=subreg(xmin1,xmax1,ymin1,ymax1,zmin1,zmax1,subregions,1)
    subregions=subreg(xmin2,xmax2,ymin2,ymax2,zmin2,zmax2,subregions,2)
    subregions=subreg(xmin3,xmax3,ymin3,ymax3,zmin3,zmax3,subregions,3)
    
    #Tamaño de la celda
    cell=(celda,celda,celda)
    
    #Creo el mesh
    mesh = df.Mesh(p1=(xminr, yminr, zminr), p2=(xmaxr, ymaxr, zmaxr),cell=cell, subregions=subregions)
   
    return mesh,subregions

#Función para definir la energía del sistema
#A es la energia de intercambio
#K es la constante de anisotropía
#H el campo aplicado[A/m]
#Devuelve la energia del sistema
def energia_del_sistema(A,K,sigma,sigma2,subregs,H=(0,0,0)):
    #easy axis
    u=(1,0,0)
    #Energias del sistema
    E_exchange=mm.Exchange(A=A)
    E_Zeeman=mm.Zeeman(H=H)
    E_Uniaxial_Anisotropy= mm.UniaxialAnisotropy(K=K, u=u)
    E_rkky=mm.RKKY(sigma=sigma, sigma2=sigma2, subregions=subregs)
    Demag=mm.Demag()
    # Defino el Hamiltoniano del sistema
    E= E_Zeeman + E_rkky + E_exchange +E_Uniaxial_Anisotropy  + Demag 
    return E

#Funcion para la energia del sistema entre dos FM
#Recibe como parametro los valores minimos en los ejes x,y,z de la región en general
def grafica_histeresis(xminr,yminr,zminr, espesor_ferromagnetico, espesor_no_magnetico,A,K,sigma,sigma2,Ms,campo_aplicado):
    #Dibujo la primera región
    mesh,subregions=region(xminr,yminr,zminr,espesor_ferromagnetico,espesor_no_magnetico)
    #Subregiones para el primer material FM (region1) y el segundo FM(region3). region2 es la region donde se ubica el material no magnetico.
    subregs=['region1', 'region3']
    #Norma de para definir la magnetizacion inicial
    norm = {'region1': Ms, 'region3': Ms, 'region2': 0}
    #Valores de la energia de intercambio para distintas regiones
    A_ = {'region1' : A  , 'region3':A, 'region2' :0}
    #Valores de la energia anisotropica para distintas regiones
    K_ = {'region1': K,'region3': K, 'region2': 0}
    #Defino la energina del sistema
    system = mm.System(name='Energia')
    system.energy=energia_del_sistema(A_,K_,sigma,sigma2,subregs)
    #Funcion para generar espines aleatorios
    def m_fun(pos):
        return [2 * random.random() - 1 for i in range(3)]
    #Se establece el sistema inicial con la energia y la magnetizacion inicial.
    system.m = df.Field(mesh, dim=3,  value=m_fun, norm=norm)
    #print('antes')
    
    #Para observar la configuracion de espines antes de minimizar la energía dle sistema
    system.m.plane('y').mpl()
    #Se minimiza la energía del sistema
    md.drive(system)    
    #print('después')
    #Para observar la configuracion de espines despues de minimizar la energía dle sistema
    system.m.plane('y').mpl()

    #Aplicamos un campo magnetico externo (A/m)    
    Hz_vacio=campo_aplicado/mm.consts.mu0
    #Vector campo aplicado para un valor minimo y maximo
    Hmin=(0,-Hz_vacio,0)
    Hmax=(0,Hz_vacio,0)
    #numero de ciclos que hace el programa
    n=85
    #Realización de la curva de histéresis
    hd.drive(system, Hmin=Hmin, Hmax=Hmax, n=n)
    #Se observan los espines desúes de finalizar la curva
    system.m.plane('y').mpl()
    return system

#Función para guardar los datos obtenidos de la curva de histeresis
#systema es el sistema que estamos guardando
#nombre es el nombre del archivo donde se guardarán los datos
#No devuelve nada
def guardar_datos(systema,nombre):
    #Campo magnetico en el eje y [mT]
    By_hysteresis=systema.table.data['By_hysteresis']
    #Magnetización del sistema
    my=systema.table.data['my']
    #Energía total del sistema
    E_t=systema.table.data['E'] 
    #Energía RKKY
    E_rkky=systema.table.data['E_rkky']
    #nombre del archivo
    n=nombre + '.txt'
    #Se crea el archivo y se escribe en este
    f=open(n,'w')
    f.write('By_hysteresis;my;ERKKY;E\n')
    for i in range(len(my)):
        f.write('{};{};{};{}\n'.format(By_hysteresis[i],my[i],E_rkky[i],E_t[i]))
    #Se cierra el archivo    
    f.close()

In [3]:
#Parametros de la region en general. Inicio la region en cero
xminr=0
yminr=0
zminr=0
#Energia de intercambio para el Cobalto 
A=30e-12
#Constante de anisotropía
K=5.2e5
#Saturación de magnetización para el Cobalto
Ms=1.4e6 #A/m

sigma=-np.sqrt(A*K/2)
sigma2=sigma

#Campo aplicado. Viene dado en teslas
campo_aplicado=0.4#T

#Espesores para el Cobalto y Niobio respectivamente
espesor_ferromagnetico=5e-9
espesor_no_magnetico=0.5e-9

##Creación de la región con los parametros dados
#mesh,subregions=region(xminr,yminr,zminr, espesor_ferromagnetico, espesor_no_magnetico)

##Para realizar la curva de histeresis se llama la función grafica_histeresis
#grafica_histeresis(xminr,yminr,zminr, espesor_ferromagnetico, espesor_no_magnetico,A,K,sigma,sigma2,Ms,campo_aplicado)